In [1]:
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

import numpy as np
import pandas as pd

In [2]:
Chung_buk = pd.read_csv('mini_project_1/Chung_nam.csv')
Chung_buk

,Unnamed: 0,업종,업소명,주요품목,가격,연락처,주소
0,0,목욕업,개나리휴사우나,목욕(대인),5000원,043-882-3337,충청북도 음성군 대소면 대금로 410-5
1,1,이미용업,수냐미용실,커트(학생),7000원,043-422-1558,충청북도 단양군 충북 단양군 매포읍 평동 3로 10
2,2,한식_일반,맛집소백산,도토리묵밥,8000원,043-423-0687,충청북도 단양군 충북 단양군 단양읍 상진2길 55
3,3,한식_면류,금주네가야밀면,물밀면,7000원,043-851-6553,충청북도 충주시 충주시 행정10길 15(목행동)
4,4,한식_일반,백세보리밥(시니어클럽),보리밥,6500원,043-855-7402,충청북도 충주시 충주시 으뜸로 9(금릉동)
...,...,...,...,...,...,...,...
316,316,한식_면류,홍천식당,메밀칼국수,7000원,043-851-2075,충청북도 충주시 교동1길15-7
317,317,한식_일반,뜨럭,청국장,5000원,043-855-1213,충청북도 충주시 교동5길4
318,318,이미용업,컷트마당,커트,5000원,043-293-9123,충청북도 청주시 상당구 월평로 206(용암동)
319,319,이미용업,고려미용실,커트,7000원,043-274-6207,충청북도 청주시 서원구 모충로 98-1(모충동)


In [3]:
Chung_buk.drop('Unnamed: 0',axis = 1, inplace = True)

Chung_buk['주소_구,군'] = Chung_buk['주소'].apply(lambda x:x.split(' ')[0])

Chung_buk['주소_구,군'] = Chung_buk['주소'].apply(lambda x:x.replace('\xa0',' '))
Chung_buk['keyword'] = Chung_buk['주소_구,군']

Chung_buk_address_list = Chung_buk['keyword'].apply(lambda x:x.split(' '))

new_Chung_buk_address_list = []

for i,item in enumerate(Chung_buk_address_list):
    #print(ad_li[i][:2])
    address = item[0] + ' ' + item[1]
    new_Chung_buk_address_list.append(address)
    
new_Chung_buk_address_list

Chung_buk['keyword'] = new_Chung_buk_address_list

Chung_buk['keyword'] = Chung_buk['keyword'] + ' ' + Chung_buk['업소명']

In [5]:
# 카카오맵 url 가져오기
url = 'https://map.kakao.com/'

# 크롬 드라이버 실행하기
driver = webdriver.Chrome()

# 카카오맵 url로 이동하기
driver.get(url)

score_list = []

for key in Chung_buk['keyword'][:321]:
    # 검색창 클릭하기
    search = driver.find_element(By.CSS_SELECTOR, '#search\.keyword\.query')

    # keyword의 key를 검색창에 넣기
    search.send_keys(key)

    # 엔터키로 이동하기
    search.send_keys(Keys.RETURN)

    time.sleep(2)

    # 웹 스크립트 받아오기
    html = driver.page_source

    # 웹스크립트에서 원하는 콘텐츠 추출하기 - parsing
    soup = BeautifulSoup(html, 'html.parser')
    
    # keyword별 평점 가져오기
    try:
        # 평점 path 가져오기
        score_path = '#info\.search\.place\.list > li:nth-child(1) > div.rating.clickArea'
        
        # score_path를 split한 후 3번째 문자 추출하기
        score = soup.select(score_path)[0].text.split('\n')[3]
        
        # 평점을 빈 리스트 L에 추가하기
        score_list.append(score)
        
    except:
        score_list.append('별점 없음')
        
    search.clear()
    
score_list

Chung_buk['평점'] = score_list
Chung_buk.drop('주소_구,군',axis = 1, inplace = True)
Chung_buk.drop('keyword',axis = 1, inplace = True)
Chung_buk.to_csv('Chung_bukscore.csv', encoding='utf-8-sig')

<>:14: SyntaxWarning: invalid escape sequence '\.'
<>:33: SyntaxWarning: invalid escape sequence '\.'
<>:14: SyntaxWarning: invalid escape sequence '\.'
<>:33: SyntaxWarning: invalid escape sequence '\.'
C:\Users\vvv\AppData\Local\Temp\ipykernel_13160\1166733382.py:14: SyntaxWarning: invalid escape sequence '\.'
  search = driver.find_element(By.CSS_SELECTOR, '#search\.keyword\.query')
C:\Users\vvv\AppData\Local\Temp\ipykernel_13160\1166733382.py:33: SyntaxWarning: invalid escape sequence '\.'
  score_path = '#info\.search\.place\.list > li:nth-child(1) > div.rating.clickArea'
